In [1]:
# 필요한 라이브러리 설치 (없는 경우에만 설치됨)
import sys
import subprocess

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# 필요한 패키지 리스트
packages = [
    "requests", 
    "beautifulsoup4", 
    "pandas", 
    "numpy"
]

# 패키지 설치
for package in packages:
    try:
        __import__(package)
        print(f"{package} 이미 설치되어 있습니다.")
    except ImportError:
        print(f"{package} 설치 중...")
        install_package(package)
        print(f"{package} 설치 완료!")
        
print("모든 필요한 라이브러리가 설치되었습니다.")

requests 이미 설치되어 있습니다.
beautifulsoup4 설치 중...


beautifulsoup4 설치 완료!
pandas 이미 설치되어 있습니다.
numpy 이미 설치되어 있습니다.
모든 필요한 라이브러리가 설치되었습니다.


In [5]:
# 필요한 라이브러리 임포트
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import os
import json
import urllib.parse

def get_news_content(url):
    """
    뉴스 URL에서 본문 내용을 추출하는 함수
    
    Args:
        url (str): 뉴스 기사 URL
        
    Returns:
        str: 추출된 뉴스 본문
    """
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'
    }
    
    try:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 네이버 뉴스 본문 선택자
        content_element = soup.select_one("#dic_area")
        if content_element:
            # 불필요한 요소 제거
            for tag in content_element.select('.end_photo_org, script'):
                tag.decompose()
            content = content_element.get_text().strip()
            return re.sub(r'\s+', ' ', content).strip()
        
        # 다른 본문 선택자 시도
        content_element = soup.select_one("#newsct_article, .newsct_article, article")
        if content_element:
            content = content_element.get_text().strip()
            return re.sub(r'\s+', ' ', content).strip()
            
        return "본문을 추출할 수 없습니다."
    
    except Exception as e:
        return f"오류 발생: {str(e)}"

def scrape_naver_news_with_link(url):
    """
    직접 뉴스 URL에서 정보를 추출하는 함수
    
    Args:
        url (str): 뉴스 기사 URL
        
    Returns:
        dict: 뉴스 정보 딕셔너리
    """
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'
    }
    
    try:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 제목 추출
        title_element = soup.select_one(".media_end_head_headline, h2.end_tit, h3.tit_headline, .sds-comps-text-type-headline1")
        title = title_element.get_text().strip() if title_element else "제목 없음"
        
        # 언론사 추출
        press_element = soup.select_one(".media_end_head_top_logo_link img, .press_logo img")
        press = press_element.get('alt', '') if press_element else ""
        
        if not press:
            press_element = soup.select_one(".media_end_head_top_logo_link, .press_logo")
            press = press_element.get_text().strip() if press_element else ""
        
        # 날짜 추출
        date_element = soup.select_one(".media_end_head_info_datestamp_time, .info_group .time, .article_info .date")
        date = date_element.get_text().strip() if date_element else ""
        
        # 본문 추출
        content_element = soup.select_one("#dic_area, #newsct_article")
        if content_element:
            # 불필요한 요소 제거
            for tag in content_element.select('.end_photo_org, script'):
                tag.decompose()
            
            content = content_element.get_text().strip()
            content = re.sub(r'\s+', ' ', content).strip()
        else:
            content = "본문을 추출할 수 없습니다."
        
        # 스니펫 생성
        snippet_element = soup.select_one(".sds-comps-text-type-body1, meta[property='og:description']")
        if snippet_element and snippet_element.name == 'meta':
            snippet = snippet_element.get('content', '')
        elif snippet_element:
            snippet = snippet_element.get_text().strip()
        else:
            snippet = content[:150] + "..." if len(content) > 150 else content
        
        return {
            'title': title,
            'link': url,
            'snippet': snippet,
            'press': press,
            'date': date,
            'content': content
        }
        
    except Exception as e:
        print(f"뉴스 추출 중 오류 발생: {e}")
        return None

def search_naver_news_by_keyword(keyword, limit=10):
    """
    네이버 뉴스에서 키워드로 검색하여 뉴스 목록을 가져오는 함수
    
    Args:
        keyword (str): 검색할 키워드
        limit (int): 검색할 최대 뉴스 개수
        
    Returns:
        DataFrame: 뉴스 정보가 담긴 데이터프레임
    """
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'
    }
    
    news_list = []
    page = 1
    encoded_keyword = urllib.parse.quote(keyword)
    collected_count = 0
    
    print(f"키워드 '{keyword}'로 뉴스 검색 중...")
    
    while collected_count < limit:
        start = (page - 1) * 10 + 1
        search_url = f"https://search.naver.com/search.naver?where=news&query={encoded_keyword}&start={start}"
        
        try:
            print(f"검색 페이지 {page} 로딩 중...")
            response = requests.get(search_url, headers=headers)
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # 네이버 뉴스 링크 찾기
            links = soup.select("a.lu8Lfh20c9DvvP05mqBf, a[href*='news.naver.com']")
            
            if not links:
                print("더 이상 뉴스 링크를 찾을 수 없습니다.")
                break
                
            print(f"페이지 {page}에서 {len(links)}개의 뉴스 링크 발견")
            
            # 중복 없이 링크만 추출
            unique_links = set()
            for link in links:
                href = link.get('href', '')
                if 'news.naver.com' in href and href not in unique_links:
                    unique_links.add(href)
            
            print(f"중복 제거 후 {len(unique_links)}개의 고유 링크 발견")
            
            # 링크에서 뉴스 정보 추출
            for href in unique_links:
                if collected_count >= limit:
                    print(f"목표한 뉴스 개수({limit}개)에 도달했습니다.")
                    break
                    
                print(f"[{collected_count+1}/{limit}] 뉴스 정보 추출 중: {href}")
                news_info = scrape_naver_news_with_link(href)
                
                if news_info:
                    collected_count += 1
                    news_list.append(news_info)
                    print(f"  - 제목: {news_info['title']}")
                    
                    # 과도한 요청 방지
                    if collected_count < limit:
                        print("  - 다음 뉴스 처리 전 1초 대기...")
                        time.sleep(1)
            
            # 이미 충분한 뉴스를 수집했으면 전체 루프 중단
            if collected_count >= limit:
                print(f"목표한 뉴스 개수({limit}개)를 모두 수집했습니다.")
                break
                
            # 다음 페이지 확인 전 대기
            print(f"다음 페이지 검색 전 2초 대기 중...")
            time.sleep(2)
            page += 1
                
        except Exception as e:
            print(f"검색 중 오류 발생: {e}")
            import traceback
            traceback.print_exc()
            break
    
    print(f"총 {len(news_list)}개의 뉴스를 수집했습니다.")
    return pd.DataFrame(news_list)

def save_results(df, output_file, format='csv'):
    """
    결과를 저장하는 함수
    
    Args:
        df (DataFrame): 저장할 데이터프레임
        output_file (str): 출력 파일 경로
        format (str): 출력 형식 (csv 또는 json)
    """
    if not output_file:
        output_file = "news"
    
    # 확장자 확인 및 추가
    base, ext = os.path.splitext(output_file)
    if not ext:
        output_file = f"{output_file}.{format}"
    
    # 형식에 따라 저장
    if format == 'json' or output_file.endswith('.json'):
        df.to_json(output_file, orient='records', force_ascii=False, indent=4)
    else:
        df.to_csv(output_file, index=False)
    
    print(f"'{output_file}'로 결과가 저장되었습니다.")

# 사용자 입력 받기
keyword = input("검색할 키워드를 입력하세요: ").strip()
if not keyword:
    print("키워드가 입력되지 않았습니다. 기본값 '인공지능'으로 설정합니다.")
    keyword = "인공지능"

# 뉴스 개수 입력
limit_str = input("수집할 최대 뉴스 개수를 입력하세요 (기본값: 10): ").strip()
try:
    limit = int(limit_str) if limit_str else 10
    if limit <= 0:
        print("뉴스 개수는 1 이상이어야 합니다. 기본값 10로 설정합니다.")
        limit = 10
except ValueError:
    print("올바른 숫자가 아닙니다. 기본값 10로 설정합니다.")
    limit = 10

# 저장 형식 입력
format_str = input("저장 형식을 선택하세요 (csv 또는 json, 기본값: csv): ").strip().lower()
format_type = format_str if format_str in ['csv', 'json'] else 'csv'

# 출력 파일명 입력
output_file = input(f"저장할 파일명을 입력하세요 (기본값: {keyword}_news.{format_type}): ").strip()
if not output_file:
    output_file = f"{keyword}_news.{format_type}"

# 뉴스 수집 시작
print(f"\n키워드 '{keyword}'로 최대 {limit}개의 뉴스를 수집합니다...")
news_df = search_naver_news_by_keyword(keyword, limit)

if news_df.empty:
    print("뉴스를 찾을 수 없습니다.")
else:
    # 결과 저장
    save_results(news_df, output_file, format_type)
    
    # 뉴스 요약 출력
    print("\n====== 수집된 뉴스 ======")
    for i, (_, row) in enumerate(news_df.iterrows()):
        print(f"{i+1}. {row['title']} ({row['press']}, {row['date']})")
        print(f"   - 요약: {row['snippet']}")
        print(f"   - 링크: {row['link']}")
        print(f"   - 본문 일부: {row['content'][:100]}...")
        print("")
        
    # 결과 데이터프레임 확인
    display(news_df.head())


키워드 '인공지능'로 최대 10개의 뉴스를 수집합니다...
키워드 '인공지능'로 뉴스 검색 중...
검색 페이지 1 로딩 중...
페이지 1에서 67개의 뉴스 링크 발견
중복 제거 후 12개의 고유 링크 발견
[1/10] 뉴스 정보 추출 중: https://n.news.naver.com/mnews/article/001/0015357393?sid=102
  - 제목: 재판에 AI 기술 도입 논의한다…사법부 인공지능위원회 출범
  - 다음 뉴스 처리 전 1초 대기...
[2/10] 뉴스 정보 추출 중: https://news.naver.com/main/static/channelPromotion.html
  - 제목: 제목 없음
  - 다음 뉴스 처리 전 1초 대기...
[3/10] 뉴스 정보 추출 중: https://n.news.naver.com/mnews/article/366/0001073043?sid=101
  - 제목: NH투자증권, 인공지능이 투자해 주는 ‘퇴직연금 로보어드바이저 일임서비스’ 출시
  - 다음 뉴스 처리 전 1초 대기...
[4/10] 뉴스 정보 추출 중: https://n.news.naver.com/mnews/article/277/0005585026?sid=102
  - 제목: 'AI 기술' 재판 업무에 활용…법원행정처 산하 인공지능위원회 출범
  - 다음 뉴스 처리 전 1초 대기...
[5/10] 뉴스 정보 추출 중: https://n.news.naver.com/mnews/article/011/0004479462?sid=102
  - 제목: AI 재판 현실되나…대법 '사법부 인공지능위원회' 출범
  - 다음 뉴스 처리 전 1초 대기...
[6/10] 뉴스 정보 추출 중: https://n.news.naver.com/mnews/article/003/0013210449?sid=105
  - 제목: 기업 리더 3명 중 1명 "AI 비서 쓰면 인력 조정 고려할 것'"
  - 다음 뉴스 처리 전 1초 대기...
[7/10] 뉴스 정보 추출 중:

,title,link,snippet,press,date,content
0,재판에 AI 기술 도입 논의한다…사법부 인공지능위원회 출범,https://n.news.naver.com/mnews/article/001/001...,위원장 '정보기술 전문가' 이숙연 대법관…AI 기반 사법정보화 시스템 구축 목표 대...,,2025.04.28. 오후 6:06,위원장 '정보기술 전문가' 이숙연 대법관…AI 기반 사법정보화 시스템 구축 목표(서...
1,제목 없음,https://news.naver.com/main/static/channelProm...,본문을 추출할 수 없습니다.,,,본문을 추출할 수 없습니다.
2,"NH투자증권, 인공지능이 투자해 주는 ‘퇴직연금 로보어드바이저 일임서비스’ 출시",https://n.news.naver.com/mnews/article/366/000...,NH투자증권이 퇴직연금 로보어드바이저 일임서비스를 출시했다고 28일 밝혔다. 퇴직연...,,2025.04.28. 오후 4:35,NH투자증권이 퇴직연금 로보어드바이저 일임서비스를 출시했다고 28일 밝혔다.퇴직연금...
3,'AI 기술' 재판 업무에 활용…법원행정처 산하 인공지능위원회 출범,https://n.news.naver.com/mnews/article/277/000...,대법원 법원행정처가 인공지능(AI) 기술을 재판 업무에 활용하는 방안을 논의하는 위...,,2025.04.28. 오후 7:12,대법원 법원행정처가 인공지능(AI) 기술을 재판 업무에 활용하는 방안을 논의하는 위...
4,AI 재판 현실되나…대법 '사법부 인공지능위원회' 출범,https://n.news.naver.com/mnews/article/011/000...,대한민국 법원이 인공지능(AI) 기술을 활용해 재판을 보다 신속하고 공정하게 만들기...,,2025.04.28. 오후 6:29,재판 지원 AI 플랫폼 개발 본격화30일 ‘재판지원 AI 플랫폼’ 설명회 개최[서울...


###pyGUI예제

In [6]:
# PyAutoGUI 패키지 설치
import sys
import subprocess

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

try:
    import pyautogui
    print("PyAutoGUI 이미 설치되어 있습니다.")
except ImportError:
    print("PyAutoGUI 설치 중...")
    install_package("pyautogui")
    print("PyAutoGUI 설치 완료!")

# PyAutoGUI 임포트 및 기본 화면 정보 출력
import pyautogui
import time

# 안전 기능 활성화 (화면 모서리로 마우스 이동 시 예외 발생 방지)
pyautogui.FAILSAFE = True

# 화면 크기 확인
screen_width, screen_height = pyautogui.size()
print(f"화면 크기: {screen_width}x{screen_height}")

PyAutoGUI 이미 설치되어 있습니다.
화면 크기: 2560x1440


In [7]:
# 화면 중앙으로 마우스 이동
def move_to_center():
    # 화면 크기 가져오기
    width, height = pyautogui.size()
    
    # 화면 중앙 좌표 계산
    center_x = width // 2
    center_y = height // 2
    
    print(f"화면 중앙 좌표: ({center_x}, {center_y})")
    
    # 1초 대기 후 화면 중앙으로 이동 (움직임을 볼 수 있도록 천천히 이동)
    print("3초 후 화면 중앙으로 마우스를 이동합니다...")
    time.sleep(3)
    
    # duration은 이동 시간(초)
    pyautogui.moveTo(center_x, center_y, duration=1)
    print("마우스가 화면 중앙으로 이동되었습니다.")
    
    # 현재 마우스 위치 출력
    current_x, current_y = pyautogui.position()
    print(f"현재 마우스 위치: ({current_x}, {current_y})")

# 함수 실행
move_to_center()

화면 중앙 좌표: (1280, 720)
3초 후 화면 중앙으로 마우스를 이동합니다...
마우스가 화면 중앙으로 이동되었습니다.
현재 마우스 위치: (1280, 720)


In [8]:
# 특정 좌표에서 우클릭하기
def right_click_at_position(x=500, y=300):
    print(f"3초 후 좌표 ({x}, {y})로 이동하여 우클릭합니다...")
    time.sleep(3)
    
    # 지정된 좌표로 이동
    pyautogui.moveTo(x, y, duration=1)
    
    # 현재 위치에서 우클릭
    pyautogui.rightClick()
    print(f"좌표 ({x}, {y})에서 우클릭을 수행했습니다.")
    
    # ESC 키를 눌러 메뉴 닫기 (필요시)
    time.sleep(1)
    pyautogui.press('esc')

# 함수 실행
right_click_at_position(500, 300)

# 사용자 정의 좌표로 실행하려면 아래 줄의 주석을 해제하고 수정하세요
# right_click_at_position(x=600, y=400)

3초 후 좌표 (500, 300)로 이동하여 우클릭합니다...
좌표 (500, 300)에서 우클릭을 수행했습니다.


In [9]:
# 특정 영역에서 파일 드래그앤드롭
def drag_and_drop_file(start_x=500, start_y=300, end_x=700, end_y=500):
    # 시작점과 끝점 출력
    print(f"시작 좌표: ({start_x}, {start_y})")
    print(f"끝 좌표: ({end_x}, {end_y})")
    
    print("3초 후 드래그앤드롭을 시작합니다...")
    time.sleep(3)
    
    # 시작 위치로 이동
    pyautogui.moveTo(start_x, start_y, duration=1)
    
    # 마우스 버튼 누르기
    pyautogui.mouseDown()
    print("마우스 버튼을 눌렀습니다.")
    
    # 0.5초 대기 (드래그 인식을 위한 대기)
    time.sleep(0.5)
    
    # 목표 위치로 드래그
    pyautogui.moveTo(end_x, end_y, duration=2)
    print(f"좌표 ({end_x}, {end_y})로 드래그 중...")
    
    # 마우스 버튼 떼기
    pyautogui.mouseUp()
    print("마우스 버튼을 떼었습니다. 드래그앤드롭 완료!")

# 함수 실행 (파일 탐색기나 바탕화면에서 실행하는 것이 좋습니다)
# drag_and_drop_file()

# 사용자 정의 좌표로 실행
drag_and_drop_file(start_x=400, start_y=300, end_x=800, end_y=500)

# 파일을 휴지통으로 드래그하는 예시 (좌표는 화면에 따라 다를 수 있음)
# 스크린 좌측 하단에 파일이 있다고 가정, 우측 하단에 휴지통이 있다고 가정
def drag_to_trash():
    # 화면 크기 가져오기
    width, height = pyautogui.size()
    
    # 좌표 계산 (예시)
    file_x = width // 4
    file_y = height - 100
    trash_x = width - 50
    trash_y = height - 50
    
    print(f"파일 위치: ({file_x}, {file_y})")
    print(f"휴지통 위치: ({trash_x}, {trash_y})")
    
    print("5초 후 파일을 휴지통으로 드래그합니다...")
    time.sleep(5)
    
    # 드래그앤드롭 수행
    pyautogui.moveTo(file_x, file_y, duration=1)
    pyautogui.mouseDown()
    time.sleep(0.5)
    pyautogui.moveTo(trash_x, trash_y, duration=2)
    pyautogui.mouseUp()
    print("파일을 휴지통으로 이동했습니다.")

# 휴지통 드래그 예시는 실행 전 주석 해제
# drag_to_trash()

시작 좌표: (400, 300)
끝 좌표: (800, 500)
3초 후 드래그앤드롭을 시작합니다...
마우스 버튼을 눌렀습니다.
좌표 (800, 500)로 드래그 중...
마우스 버튼을 떼었습니다. 드래그앤드롭 완료!


In [10]:
# 웹페이지 천천히 스크롤
def slow_scroll_webpage(scroll_amount=300, scroll_count=5, delay=1):
    print("3초 후 스크롤을 시작합니다...")
    print("웹 브라우저가 포커스되어 있는지 확인하세요!")
    time.sleep(3)
    
    print(f"총 {scroll_count}회 스크롤 예정, 각 스크롤 간격: {delay}초")
    
    # 스크롤 수행
    for i in range(scroll_count):
        print(f"스크롤 {i+1}/{scroll_count} 수행 중...")
        
        # 스크롤 다운 (양수 값은 아래로 스크롤)
        pyautogui.scroll(-scroll_amount)
        
        # 스크롤 간 대기
        time.sleep(delay)
    
    print("스크롤 완료!")

# 기본 설정으로 실행
slow_scroll_webpage()

# 사용자 정의 값으로 실행 (더 많은 스크롤)
# slow_scroll_webpage(scroll_amount=200, scroll_count=10, delay=0.5)

# 페이지를 부드럽게 스크롤하는 대안 함수
def smooth_scroll_webpage(total_scroll=2000, steps=20, delay=0.1):
    print("3초 후 부드러운 스크롤을 시작합니다...")
    print("웹 브라우저가 포커스되어 있는지 확인하세요!")
    time.sleep(3)
    
    # 한 번에 스크롤할 양 계산
    scroll_per_step = total_scroll // steps
    
    print(f"총 {steps}단계로 {total_scroll}픽셀 스크롤 예정")
    
    # 부드러운 스크롤 수행
    for i in range(steps):
        print(f"스크롤 단계 {i+1}/{steps} 수행 중...")
        
        # 한 단계씩 스크롤
        pyautogui.scroll(-scroll_per_step)
        
        # 매우 짧은 대기 시간
        time.sleep(delay)
    
    print("부드러운 스크롤 완료!")

# 부드러운 스크롤 실행 (원하는 경우 주석 해제)
# smooth_scroll_webpage()

3초 후 스크롤을 시작합니다...
웹 브라우저가 포커스되어 있는지 확인하세요!
총 5회 스크롤 예정, 각 스크롤 간격: 1초
스크롤 1/5 수행 중...
스크롤 2/5 수행 중...
스크롤 3/5 수행 중...
스크롤 4/5 수행 중...
스크롤 5/5 수행 중...
스크롤 완료!


In [12]:
# 필요한 라이브러리 설치 확인 및 임포트
import sys
import subprocess
import time

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

try:
    import pyautogui
    print("PyAutoGUI 이미 설치되어 있습니다.")
except ImportError:
    print("PyAutoGUI 설치 중...")
    install_package("pyautogui")
    import pyautogui
    print("PyAutoGUI 설치 완료!")

# 0.5초 간격으로 이메일 주소 입력하는 함수
def type_email_with_delay(email="example@email.com", delay=0.5):
    print(f"3초 후 '{email}' 이메일 주소를 {delay}초 간격으로 입력합니다...")
    print("입력 필드에 커서를 위치시키세요!")
    
    # 사용자가 입력 필드에 커서를 위치시킬 시간 제공
    for i in range(3, 0, -1):
        print(f"{i}...")
        time.sleep(1)
    
    print("입력 시작!")
    
    # 각 문자를 delay 간격으로 입력
    pyautogui.write(email, interval=delay)
    
    print(f"이메일 주소 '{email}'가 입력되었습니다.")

# 기본 이메일 주소로 함수 실행
type_email_with_delay()

# 사용자 정의 이메일과 딜레이로 실행하려면 아래 코드의 주석을 해제하고 값을 수정하세요
# type_email_with_delay(email="your.email@domain.com", delay=0.5)

# 여러 이메일 주소를 입력하는 함수 예시
def type_multiple_emails(emails=["user1@example.com", "user2@example.com", "user3@example.com"], 
                         delay_between_chars=0.5, 
                         delay_between_emails=2):
    print(f"3초 후 {len(emails)}개의 이메일 주소를 입력합니다...")
    print("입력 필드에 커서를 위치시키세요!")
    
    for i in range(3, 0, -1):
        print(f"{i}...")
        time.sleep(1)
    
    for i, email in enumerate(emails):
        print(f"이메일 {i+1}/{len(emails)} 입력 중: {email}")
        
        # 현재 이메일 입력
        pyautogui.write(email, interval=delay_between_chars)
        
        # Enter 키 입력 또는 다음 입력 필드로 이동 (Tab)
        if i < len(emails) - 1:
            print(f"{delay_between_emails}초 대기 후 다음 이메일 입력...")
            pyautogui.press('tab')  # 다음 필드로 이동 (또는 'enter'를 사용할 수도 있음)
            time.sleep(delay_between_emails)
    
    print("모든 이메일 주소 입력 완료!")

# 여러 이메일 입력 예시 - 필요한 경우 주석 해제
# type_multiple_emails()

PyAutoGUI 이미 설치되어 있습니다.
3초 후 'example@email.com' 이메일 주소를 0.5초 간격으로 입력합니다...
입력 필드에 커서를 위치시키세요!
3...
2...
1...
입력 시작!
이메일 주소 'example@email.com'가 입력되었습니다.


example@email.com

In [ ]:
# 필요한 라이브러리 설치 확인 및 임포트
import sys
import subprocess
import time
from datetime import datetime
import os

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

try:
    import pyautogui
    print("PyAutoGUI 이미 설치되어 있습니다.")
except ImportError:
    print("PyAutoGUI 설치 중...")
    install_package("pyautogui")
    import pyautogui
    print("PyAutoGUI 설치 완료!")

def take_screenshot_with_timestamp():
    # 현재 날짜와 시간을 파일명 형식으로 가져오기
    current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"screenshot_{current_time}.png"
    
    # 저장 경로 설정 (기본값은 현재 작업 디렉터리)
    # 원하는 경로로 변경 가능
    save_path = os.path.join(os.getcwd(), filename)
    
    print(f"3초 후 스크린샷을 찍습니다...")
    for i in range(3, 0, -1):
        print(f"{i}...")
        time.sleep(1)
    
    # 스크린샷 찍기
    screenshot = pyautogui.screenshot()
    
    # 파일로 저장
    screenshot.save(save_path)
    
    print(f"스크린샷을 '{save_path}'에 저장했습니다.")
    return save_path

# 함수 실행
screenshot_path = take_screenshot_with_timestamp()

In [15]:
# 필요한 라이브러리 설치 확인 및 임포트
import sys
import subprocess
import time
from datetime import datetime
import os

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

try:
    import pyautogui
    print("PyAutoGUI 이미 설치되어 있습니다.")
except ImportError:
    print("PyAutoGUI 설치 중...")
    install_package("pyautogui")
    import pyautogui
    print("PyAutoGUI 설치 완료!")

def take_screenshot_with_timestamp():
    # 현재 날짜와 시간을 파일명 형식으로 가져오기
    current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"screenshot_{current_time}.png"
    
    # 저장 경로 설정 (기본값은 현재 작업 디렉터리)
    # 원하는 경로로 변경 가능
    save_path = os.path.join(os.getcwd(), filename)
    
    print(f"3초 후 스크린샷을 찍습니다...")
    for i in range(3, 0, -1):
        print(f"{i}...")
        time.sleep(1)
    
    # 스크린샷 찍기
    screenshot = pyautogui.screenshot()
    
    # 파일로 저장
    screenshot.save(save_path)
    
    print(f"스크린샷을 '{save_path}'에 저장했습니다.")
    return save_path

# 함수 실행
screenshot_path = take_screenshot_with_timestamp()

PyAutoGUI 이미 설치되어 있습니다.
3초 후 스크린샷을 찍습니다...
3...
2...
1...
스크린샷을 '/Users/deck/Downloads/파이썬자동화 강의교안/screenshot_20250429_000327.png'에 저장했습니다.


In [ ]:
# 필요한 라이브러리 설치 확인 및 임포트
import sys
import subprocess
import time
import os

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

try:
    import pyautogui
    print("PyAutoGUI 이미 설치되어 있습니다.")
except ImportError:
    print("PyAutoGUI 설치 중...")
    install_package("pyautogui")
    import pyautogui
    print("PyAutoGUI 설치 완료!")

def find_and_click_button(button_image_path, confidence=0.7):
    """
    화면에서 특정 이미지를 찾아 클릭하는 함수
    
    Args:
        button_image_path (str): 찾을 버튼 이미지의 파일 경로
        confidence (float): 이미지 일치 신뢰도 (0~1, 높을수록 더 정확히 일치)
    
    Returns:
        bool: 클릭 성공 여부
    """
    if not os.path.exists(button_image_path):
        print(f"오류: 이미지 파일을 찾을 수 없습니다 - {button_image_path}")
        return False
    
    print(f"화면에서 '{button_image_path}' 이미지를 찾는 중...")
    
    try:
        # 화면에서 이미지 찾기
        button_location = pyautogui.locateOnScreen(button_image_path, confidence=confidence)
        
        if button_location is None:
            print("이미지를 화면에서 찾을 수 없습니다.")
            return False
        
        # 이미지 중앙 좌표 찾기
        button_point = pyautogui.center(button_location)
        print(f"이미지를 찾았습니다! 위치: {button_point}")
        
        # 이미지 위치로 마우스 이동 후 클릭
        pyautogui.moveTo(button_point.x, button_point.y, duration=1)
        pyautogui.click()
        
        print(f"좌표 ({button_point.x}, {button_point.y})에서 클릭을 수행했습니다.")
        return True
    
    except Exception as e:
        print(f"오류 발생: {e}")
        return False

# 사용 예시 (실제 실행하려면 이미지 파일 경로를 지정하고 주석을 해제하세요)
# button_image = "path/to/your/button_image.png"  # 찾을 버튼 이미지 경로 지정
# find_and_click_button(button_image)

# 이미지 사전 저장 예시 함수
def save_button_image_for_later():
    """
    현재 마우스 위치 주변의 이미지를 저장해 나중에 찾을 수 있게 하는 함수
    """
    print("3초 후 마우스 위치 주변의 버튼 이미지를 저장합니다...")
    print("마우스를 저장하려는 버튼 위에 올려놓으세요!")
    
    for i in range(3, 0, -1):
        print(f"{i}...")
        time.sleep(1)
    
    # 현재 마우스 위치 가져오기
    x, y = pyautogui.position()
    print(f"현재 마우스 위치: ({x}, {y})")
    
    # 마우스 주변 영역 스크린샷 찍기 (100x100 픽셀)
    region = (x-50, y-50, 100, 100)  # (left, top, width, height)
    button_image = pyautogui.screenshot(region=region)
    
    # 파일로 저장
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    image_path = f"button_image_{timestamp}.png"
    button_image.save(image_path)
    
    print(f"버튼 이미지를 '{image_path}'에 저장했습니다.")
    print(f"이 이미지를 find_and_click_button() 함수에 사용할 수 있습니다.")
    
    return image_path

# 필요한 경우 버튼 이미지 사전 저장 (실제 실행하려면 주석을 해제하세요)
# saved_button_image = save_button_image_for_later()
# time.sleep(3)  # 잠시 대기
# find_and_click_button(saved_button_image)  # 저장된 이미지로 찾기 및 클릭

In [16]:
# 필요한 라이브러리 설치 확인 및 임포트
import sys
import subprocess
import platform
import time

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# PyAutoGUI 설치 확인
try:
    import pyautogui
    print("PyAutoGUI 이미 설치되어 있습니다.")
except ImportError:
    print("PyAutoGUI 설치 중...")
    install_package("pyautogui")
    import pyautogui
    print("PyAutoGUI 설치 완료!")

# 운영체제 확인
os_name = platform.system()
print(f"현재 운영체제: {os_name}")

def get_all_window_titles():
    """현재 실행 중인 모든 창의 제목을 가져오는 함수"""
    
    if os_name == "Windows":
        # Windows에서는 pygetwindow 라이브러리 사용
        try:
            import pygetwindow as gw
            print("pygetwindow 이미 설치되어 있습니다.")
        except ImportError:
            print("pygetwindow 설치 중...")
            install_package("pygetwindow")
            import pygetwindow as gw
            print("pygetwindow 설치 완료!")
        
        windows = gw.getAllTitles()
        
    elif os_name == "Darwin":  # macOS
        # macOS에서는 AppKit 사용
        try:
            from AppKit import NSWorkspace
            print("AppKit을 사용할 수 있습니다.")
        except ImportError:
            print("pyobjc 설치 중... (macOS에서 AppKit 사용을 위해 필요)")
            install_package("pyobjc")
            from AppKit import NSWorkspace
            print("pyobjc 설치 완료!")
        
        running_apps = NSWorkspace.sharedWorkspace().runningApplications()
        windows = [app.localizedName() for app in running_apps if app.localizedName()]
        
    elif os_name == "Linux":
        # Linux에서는 wmctrl 명령어 또는 Xlib 사용
        try:
            import subprocess
            result = subprocess.run(['wmctrl', '-l'], capture_output=True, text=True)
            if result.returncode != 0:
                print("wmctrl이 설치되어 있지 않습니다. 설치하시려면 'sudo apt-get install wmctrl'을 실행하세요.")
                windows = ["wmctrl이 필요합니다"]
            else:
                windows = [line.split(None, 3)[3] for line in result.stdout.splitlines()]
        except Exception as e:
            print(f"오류 발생: {e}")
            windows = [f"오류: {e}"]
    else:
        windows = [f"지원되지 않는 운영체제: {os_name}"]
    
    return windows

# 실행 중인 모든 창 제목 가져오기
window_titles = get_all_window_titles()

# 결과 출력
print("\n===== 현재 실행 중인 창 목록 =====")
for i, title in enumerate(window_titles, 1):
    if title.strip():  # 빈 제목은 제외
        print(f"{i}. {title}")

print(f"\n총 {len([t for t in window_titles if t.strip()])}개의 창이 실행 중입니다.")

PyAutoGUI 이미 설치되어 있습니다.
현재 운영체제: Darwin
AppKit을 사용할 수 있습니다.

===== 현재 실행 중인 창 목록 =====
1. loginwindow
2. universalaccessd
3. WindowManager
4. Google Chrome
5. com.apple.PressAndHold
6. 한국어 IM
7. 알림 센터
8. 메시지
9. DeepL
10. Dock
11. SystemUIServer
12. 제어 센터
13. 배경화면
14. Finder
15. ViewBridgeAuxiliary
16. Adobe Content Synchronizer Finder Extension
17. UIKitSystem
18. 추가 Dock
19. Spotlight
20. universalAccessAuthWarn
21. CursorUIViewService
22. DeepLUninstall
23. DeepL Networking
24. DeepL Graphics and Media
25. DeepL 웹 콘텐츠
26. Acrobat Collaboration Synchronizer
27. TextInputMenuAgent
28. AirPlay 화면 미러링
29. Dropbox
30. RunCat
31. Clipy
32. MxNotify
33. Creative Cloud Interprocess Service
34. Dropbox와 Finder 연동
35. Creative Cloud
36. Desktop Video Updater
37. Wi-Fi
38. Creative Cloud Core Service
39. Creative Cloud Helper
40. Adobe Content Synchronizer
41. Google Chrome Helper (Plugin)
42. Acrobat
43. AcroCEF
44. Day One Helper
45. DeepL 웹 콘텐츠
46. 가족
47. familycircled
48. talagentd
49. User

In [ ]:
# 필요한 라이브러리 설치 확인 및 임포트
import sys
import subprocess
import platform
import time

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

try:
    import pyautogui
    print("PyAutoGUI 이미 설치되어 있습니다.")
except ImportError:
    print("PyAutoGUI 설치 중...")
    install_package("pyautogui")
    import pyautogui
    print("PyAutoGUI 설치 완료!")

# 운영체제 확인
os_name = platform.system()
print(f"현재 운영체제: {os_name}")

def open_notepad_and_type(text="안녕하세요! PyAutoGUI를 사용하여 자동으로 입력된 텍스트입니다."):
    """
    메모장(또는 텍스트 편집기)을 실행하고 텍스트를 자동으로 입력하는 함수
    
    Args:
        text (str): 입력할 텍스트
    """
    print("3초 후 텍스트 편집기를 실행합니다...")
    for i in range(3, 0, -1):
        print(f"{i}...")
        time.sleep(1)
    
    if os_name == "Windows":
        # Windows에서 메모장 실행
        subprocess.Popen("notepad")
        app_name = "메모장"
        
    elif os_name == "Darwin":  # macOS
        # macOS에서 TextEdit 실행
        subprocess.Popen(["open", "-a", "TextEdit"])
        app_name = "TextEdit"
        
    elif os_name == "Linux":
        # Linux에서 gedit 또는 다른 텍스트 편집기 실행
        try:
            subprocess.Popen(["gedit"])
            app_name = "gedit"
        except FileNotFoundError:1
            try:
                subprocess.Popen(["xed"])
                app_name = "xed"
            except FileNotFoundError:
                try:
                    subprocess.Popen(["kwrite"])
                    app_name = "kwrite"
                except FileNotFoundError:
                    print("지원되는 텍스트 편집기를 찾을 수 없습니다.")
                    return
    else:
        print(f"지원되지 않는 운영체제: {os_name}")
        return
    
    # 앱이 실행될 때까지 대기
    print(f"{app_name}가 실행될 때까지 3초 대기 중...")
    time.sleep(3)
    
    # 현재 날짜와 시간 추가
    from datetime import datetime
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    full_text = f"[{current_time}]\n\n{text}\n\n이 텍스트는 PyAutoGUI를 통해 자동으로 입력되었습니다."
    
    # 텍스트 입력 시작
    print("텍스트 입력을 시작합니다...")
    pyautogui.write(full_text, interval=0.05)  # 각 문자 간 0.05초 간격으로 입력
    
    print("텍스트 입력이 완료되었습니다!")
    
    # 입력한 텍스트 저장 여부 확인
    save_prompt = input("\n텍스트를 파일로 저장하시겠습니까? (y/n): ").strip().lower()
    
    if save_prompt == 'y':
        # 저장 단축키 입력
        print("단축키를 사용하여 저장 대화상자를 엽니다...")
        
        if os_name == "Windows" or os_name == "Linux":
            pyautogui.hotkey('ctrl', 's')
        elif os_name == "Darwin":  # macOS
            pyautogui.hotkey('command', 's')
        
        # 저장 대화상자가 열릴 때까지 대기
        time.sleep(1)
        
        # 파일명 입력 (현재 시간 기준)
        filename = f"pyautogui_text_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
        pyautogui.write(filename)
        
        # Enter 키를 눌러 저장
        time.sleep(0.5)
        pyautogui.press('enter')
        
        print(f"파일이 '{filename}'으로 저장되었습니다.")
    else:
        print("파일을 저장하지 않았습니다.")

# 함수 실행
open_notepad_and_type()

# 사용자 정의 텍스트로 실행하려면 아래 코드의 주석을 해제하고 텍스트를 수정하세요
# custom_text = """이것은 여러 줄로 된
# 사용자 정의 텍스트입니다.
# PyAutoGUI로 자동화된 텍스트 입력의 예제입니다."""
# open_notepad_and_type(custom_text)

PyAutoGUI 이미 설치되어 있습니다.
현재 운영체제: Darwin
3초 후 텍스트 편집기를 실행합니다...
3...
2...
1...
TextEdit가 실행될 때까지 3초 대기 중...
텍스트 입력을 시작합니다...
텍스트 입력이 완료되었습니다!


In [2]:
import sys
import subprocess
import os
import time
import threading
from datetime import datetime
from tkinter import Tk, Label, Button, Entry, StringVar, Frame, DoubleVar, Spinbox, filedialog, Text, Scrollbar, VERTICAL, RIGHT, Y, END, LEFT, BOTH, Toplevel, Canvas
from tkinter.ttk import Progressbar, Style

# 필요한 라이브러리 설치 확인
def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# 필요한 패키지 설치
try:
    import pyautogui
    from PIL import Image, ImageTk, ImageGrab
except ImportError:
    print("필요한 패키지 설치 중...")
    install_package("pyautogui")
    install_package("pillow")
    import pyautogui
    from PIL import Image, ImageTk, ImageGrab

class AreaSelector(Toplevel):
    """화면 영역 선택을 위한 투명 오버레이 창"""
    def __init__(self, parent, callback):
        super().__init__(parent)
        self.parent = parent
        self.callback = callback
        self.start_x = 0
        self.start_y = 0
        self.current_x = 0
        self.current_y = 0
        
        # 창 설정
        self.attributes("-fullscreen", True)
        self.attributes("-alpha", 0.3)  # 반투명
        self.attributes("-topmost", True)
        
        # 캔버스 생성
        self.canvas = Canvas(self, cursor="cross", bg="grey")
        self.canvas.pack(fill=BOTH, expand=True)
        
        # 선택 사각형
        self.rect = None
        
        # 이벤트 바인딩
        self.canvas.bind("<ButtonPress-1>", self.on_press)
        self.canvas.bind("<B1-Motion>", self.on_motion)
        self.canvas.bind("<ButtonRelease-1>", self.on_release)
        
        # ESC 키로 취소
        self.bind("<Escape>", self.on_cancel)
        
        self.protocol("WM_DELETE_WINDOW", self.on_cancel)
        
    def on_press(self, event):
        # 드래그 시작점 저장
        self.start_x = self.canvas.canvasx(event.x)
        self.start_y = self.canvas.canvasy(event.y)
        
        # 기존 사각형 삭제
        if self.rect:
            self.canvas.delete(self.rect)
        
        # 새 사각형 생성
        self.rect = self.canvas.create_rectangle(
            self.start_x, self.start_y, self.start_x + 1, self.start_y + 1,
            outline="red", width=2
        )
    
    def on_motion(self, event):
        # 현재 위치 업데이트
        self.current_x = self.canvas.canvasx(event.x)
        self.current_y = self.canvas.canvasy(event.y)
        
        # 사각형 크기 업데이트
        self.canvas.coords(self.rect, self.start_x, self.start_y, self.current_x, self.current_y)
    
    def on_release(self, event):
        # 최종 크기의 사각형 영역 좌표 (x1, y1, x2, y2)
        x1 = min(self.start_x, self.current_x)
        y1 = min(self.start_y, self.current_y)
        x2 = max(self.start_x, self.current_x)
        y2 = max(self.start_y, self.current_y)
        
        # 창 닫기
        self.destroy()
        
        # 선택 영역이 충분히 큰 경우에만 콜백 호출
        if abs(x2 - x1) > 10 and abs(y2 - y1) > 10:
            self.callback((int(x1), int(y1), int(x2), int(y2)))
        else:
            # 영역이 너무 작으면 취소로 처리
            self.callback(None)
    
    def on_cancel(self, event=None):
        # 선택 취소
        self.destroy()
        self.callback(None)

class ScreenshotApp:
    def __init__(self, root):
        self.root = root
        self.root.title("스크린샷 자동화 도구")
        self.root.geometry("500x600")
        self.root.resizable(True, True)
        
        # 변수 초기화
        self.selected_area = None  # (x1, y1, x2, y2)
        self.screenshot_interval = DoubleVar(value=5.0)
        self.save_folder = StringVar(value=os.path.expanduser("~/Desktop"))
        self.is_running = False
        self.thread = None
        self.stop_event = threading.Event()
        
        # GUI 스타일 설정
        self.style = Style()
        self.style.configure("TButton", font=("Arial", 10))
        self.style.configure("TLabel", font=("Arial", 10))
        
        # GUI 요소 생성
        self.create_widgets()
    
    def create_widgets(self):
        # 영역 선택 프레임
        area_frame = Frame(self.root, pady=10)
        area_frame.pack(fill='x', padx=20)
        
        Label(area_frame, text="캡처 영역:").pack(side=LEFT, padx=(0, 10))
        self.area_label = Label(area_frame, text="선택되지 않음", width=30)
        self.area_label.pack(side=LEFT, padx=(0, 10))
        
        self.select_area_btn = Button(area_frame, text="영역 선택", command=self.select_area)
        self.select_area_btn.pack(side=LEFT)
        
        # 간격 설정 프레임
        interval_frame = Frame(self.root, pady=10)
        interval_frame.pack(fill='x', padx=20)
        
        Label(interval_frame, text="캡처 간격(초):").pack(side=LEFT, padx=(0, 10))
        
        self.interval_spinbox = Spinbox(interval_frame, from_=0.1, to=3600, increment=0.1, 
                                        textvariable=self.screenshot_interval, width=10)
        self.interval_spinbox.pack(side=LEFT)
        
        # 저장 폴더 프레임
        folder_frame = Frame(self.root, pady=10)
        folder_frame.pack(fill='x', padx=20)
        
        Label(folder_frame, text="저장 폴더:").pack(side=LEFT, padx=(0, 10))
        
        self.folder_label = Label(folder_frame, textvariable=self.save_folder, width=30, anchor='w')
        self.folder_label.pack(side=LEFT, padx=(0, 10))
        
        self.browse_btn = Button(folder_frame, text="폴더 선택", command=self.browse_folder)
        self.browse_btn.pack(side=LEFT)
        
        # 미리보기 프레임
        self.preview_frame = Frame(self.root, bg='black', width=400, height=300)
        self.preview_frame.pack(padx=20, pady=10, fill=BOTH, expand=True)
        
        self.preview_label = Label(self.preview_frame, text="영역을 선택하면 미리보기가 표시됩니다",
                                  bg='black', fg='white')
        self.preview_label.pack(expand=True)
        
        # 제어 버튼 프레임
        control_frame = Frame(self.root, pady=10)
        control_frame.pack(fill='x', padx=20)
        
        self.start_btn = Button(control_frame, text="시작", bg='green', fg='white', 
                               command=self.start_screenshots, width=10)
        self.start_btn.pack(side=LEFT, padx=(0, 10))
        
        self.stop_btn = Button(control_frame, text="정지", bg='red', fg='white', 
                              command=self.stop_screenshots, width=10, state='disabled')
        self.stop_btn.pack(side=LEFT)
        
        # 상태 표시 프레임
        status_frame = Frame(self.root, pady=10)
        status_frame.pack(fill='x', padx=20)
        
        self.progress_bar = Progressbar(status_frame, orient='horizontal', length=300, mode='indeterminate')
        self.progress_bar.pack(side=LEFT, padx=(0, 10), fill='x', expand=True)
        
        self.status_label = Label(status_frame, text="준비")
        self.status_label.pack(side=LEFT)
        
        # 로그 영역
        log_frame = Frame(self.root, pady=10)
        log_frame.pack(fill='both', expand=True, padx=20)
        
        self.log_text = Text(log_frame, height=5, width=50)
        self.log_text.pack(side=LEFT, fill='both', expand=True)
        
        scrollbar = Scrollbar(log_frame, orient=VERTICAL, command=self.log_text.yview)
        scrollbar.pack(side=RIGHT, fill=Y)
        self.log_text.config(yscrollcommand=scrollbar.set)
        
        # 초기 로그 메시지
        self.log("스크린샷 자동화 도구가 시작되었습니다.")
        self.log(f"기본 저장 경로: {self.save_folder.get()}")
    
    def log(self, message):
        """로그 메시지 추가"""
        timestamp = datetime.now().strftime("%H:%M:%S")
        self.log_text.insert(END, f"[{timestamp}] {message}\n")
        self.log_text.see(END)  # 자동 스크롤
    
    def select_area(self):
        """화면에서 캡처할 영역 선택"""
        self.log("화면 영역 선택 모드로 전환합니다. 드래그하여 영역을 선택하세요.")
        
        # 메인 창 최소화
        self.root.iconify()
        time.sleep(0.5)  # 창이 최소화될 시간을 주기
        
        # 영역 선택 창 표시
        selector = AreaSelector(self.root, self.on_area_selected)
        self.root.wait_window(selector)  # 영역 선택 창이 닫힐 때까지 대기
    
    def on_area_selected(self, area):
        """영역 선택 완료 콜백"""
        self.root.deiconify()  # 메인 창 복원
        
        if area is None:
            self.log("영역 선택이 취소되었습니다.")
            return
        
        x1, y1, x2, y2 = area
        width = x2 - x1
        height = y2 - y1
        
        self.selected_area = area
        self.area_label.config(text=f"선택됨: ({x1},{y1}) - ({x2},{y2}) [{width}x{height}]")
        
        self.log(f"영역이 선택되었습니다: ({x1},{y1}) - ({x2},{y2}) [크기: {width}x{height}]")
        
        # 미리보기 생성
        self.update_preview()
    
    def update_preview(self):
        """선택한 영역의 미리보기 업데이트"""
        if self.selected_area:
            try:
                # 선택 영역의 스크린샷 찍기
                x1, y1, x2, y2 = self.selected_area
                width = x2 - x1
                height = y2 - y1
                screenshot = pyautogui.screenshot(region=(x1, y1, width, height))
                
                # 미리보기 크기에 맞게 조정
                max_width = self.preview_frame.winfo_width() - 20
                max_height = self.preview_frame.winfo_height() - 20
                
                if max_width <= 1 or max_height <= 1:  # 창이 아직 렌더링되지 않은 경우
                    max_width = 380
                    max_height = 280
                
                img_width, img_height = screenshot.size
                scale = min(max_width/img_width, max_height/img_height)
                
                if scale < 1:  # 미리보기가 원본보다 작아야 할 경우
                    new_width = int(img_width * scale)
                    new_height = int(img_height * scale)
                    screenshot = screenshot.resize((new_width, new_height), Image.LANCZOS)
                
                # Tkinter에서 표시 가능한 PhotoImage로 변환
                img = ImageTk.PhotoImage(screenshot)
                
                # 기존 미리보기 제거하고 새로운 이미지 표시
                for widget in self.preview_frame.winfo_children():
                    widget.destroy()
                
                preview_label = Label(self.preview_frame, image=img)
                preview_label.image = img  # 참조 유지
                preview_label.pack(expand=True)
                
            except Exception as e:
                self.log(f"미리보기 생성 중 오류 발생: {e}")
                # 오류 발생 시 텍스트 레이블로 대체
                for widget in self.preview_frame.winfo_children():
                    widget.destroy()
                Label(self.preview_frame, text="미리보기를 생성할 수 없습니다", 
                      bg='black', fg='white').pack(expand=True)
    
    def browse_folder(self):
        """저장 폴더 선택"""
        folder = filedialog.askdirectory(initialdir=self.save_folder.get())
        if folder:
            self.save_folder.set(folder)
            self.log(f"저장 폴더 변경: {folder}")
    
    def take_screenshot(self):
        """스크린샷 캡처 및 저장"""
        if not self.selected_area:
            self.log("오류: 캡처 영역이 선택되지 않았습니다")
            return False
        
        try:
            x1, y1, x2, y2 = self.selected_area
            width = x2 - x1
            height = y2 - y1
            
            # 스크린샷 캡처
            screenshot = pyautogui.screenshot(region=(x1, y1, width, height))
            
            # 저장 파일명 생성 (타임스탬프 포함)
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"screenshot_{timestamp}.png"
            save_path = os.path.join(self.save_folder.get(), filename)
            
            # 저장
            screenshot.save(save_path)
            self.log(f"스크린샷 저장됨: {filename}")
            return True
        
        except Exception as e:
            self.log(f"스크린샷 캡처 중 오류 발생: {e}")
            return False
    
    def screenshot_thread(self):
        """백그라운드에서 주기적으로 스크린샷 캡처"""
        count = 0
        self.log("스크린샷 자동 캡처를 시작합니다...")
        
        while not self.stop_event.is_set():
            # 스크린샷 캡처
            if self.take_screenshot():
                count += 1
                # 상태 업데이트
                self.root.after(0, self.update_status, f"캡처 중... ({count}장 완료)")
            
            # 설정된 간격만큼 대기
            interval = self.screenshot_interval.get()
            
            # 0.1초 단위로 체크하면서 대기 (즉시 중지 가능하도록)
            for _ in range(int(interval * 10)):
                if self.stop_event.is_set():
                    break
                time.sleep(0.1)
        
        # 작업 완료 후 상태 업데이트
        self.root.after(0, self.finalize_screenshots, count)
    
    def update_status(self, message):
        """UI 상태 업데이트 (메인 스레드에서 호출되어야 함)"""
        self.status_label.config(text=message)
    
    def finalize_screenshots(self, count):
        """스크린샷 작업 종료 처리"""
        self.progress_bar.stop()
        self.start_btn.config(state='normal')
        self.stop_btn.config(state='disabled')
        self.select_area_btn.config(state='normal')
        self.browse_btn.config(state='normal')
        self.interval_spinbox.config(state='normal')
        
        self.status_label.config(text=f"완료 (총 {count}장의 스크린샷)")
        self.is_running = False
        self.log(f"자동 캡처가 중지되었습니다. 총 {count}장의 스크린샷이 저장되었습니다.")
    
    def start_screenshots(self):
        """스크린샷 자동 캡처 시작"""
        if self.is_running:
            return
        
        if not self.selected_area:
            self.log("오류: 캡처 영역을 먼저 선택해주세요")
            return
        
        if not os.path.exists(self.save_folder.get()):
            self.log(f"오류: 저장 폴더가 존재하지 않습니다: {self.save_folder.get()}")
            return
        
        # 스레드 중지 이벤트 초기화
        self.stop_event.clear()
        
        # UI 상태 업데이트
        self.start_btn.config(state='disabled')
        self.stop_btn.config(state='normal')
        self.select_area_btn.config(state='disabled')
        self.browse_btn.config(state='disabled')
        self.interval_spinbox.config(state='disabled')
        
        self.progress_bar.start(10)
        self.status_label.config(text="캡처 중...")
        self.is_running = True
        
        # 스크린샷 스레드 시작
        self.thread = threading.Thread(target=self.screenshot_thread)
        self.thread.daemon = True
        self.thread.start()
        
        interval = self.screenshot_interval.get()
        self.log(f"자동 캡처가 시작되었습니다. 간격: {interval}초")
    
    def stop_screenshots(self):
        """스크린샷 자동 캡처 중지"""
        if not self.is_running:
            return
        
        self.log("자동 캡처를 중지하는 중...")
        self.status_label.config(text="중지하는 중...")
        self.stop_event.set()
        
        # 스레드가 종료될 때까지 기다리지 않음 - finalize_screenshots에서 처리

# 메인 애플리케이션 실행
if __name__ == "__main__":
    root = Tk()
    app = ScreenshotApp(root)
    root.mainloop()